In [6]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1065, -0.1907,  0.1071,  0.0712, -0.0533, -0.0039, -0.1149, -0.2018,
         -0.3443, -0.1287],
        [-0.1307, -0.0910, -0.0081,  0.0692, -0.1625,  0.0409, -0.1012, -0.1514,
         -0.2303, -0.1841]], grad_fn=<AddmmBackward0>)

In [4]:
# 在下面的代码片段中，我们从零开始编写一个块。 
# 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 
# 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。
class MLP(nn.Module):
    # 用模型声明参数层
    def __init__(self):
        # 调用MLP的父类Block的构造函数来执行必要的初始化。 这样，在类实例化时还可以指定其他函数参数，例如模型参数的访问。
        super().__init__()
        # 隐藏层
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
        
    # 定义模型的前向传播,即如何根据输入x返回所需的模型输出
    def forward(self,X):
        # 通过激活函数ReLU处理隐藏层的输出
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[-0.1300, -0.0376,  0.0751, -0.0770,  0.0246, -0.0219, -0.1027, -0.1690,
          0.0028, -0.1055],
        [-0.2843, -0.0708,  0.1549,  0.0617, -0.0703,  0.0210, -0.1860, -0.3221,
          0.0476, -0.1954]], grad_fn=<AddmmBackward0>)

In [8]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        for block in args:
            # 这里，block是Module子类的一个实例，我们把它保存在'Module'类的成员变量_children中。 
            # 这样，block就成为了MySequential类对象的一个模块，从而MySequential的参数就包括了block中的所有需要训练的参数。
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
# 下面，我们用MySequential类来实现前面描述的MLP类
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0387, -0.1209, -0.1194,  0.1890, -0.3328,  0.4535, -0.1412,  0.1038,
          0.1036,  0.1111],
        [-0.0550,  0.0205, -0.0841,  0.1580, -0.2243,  0.2520, -0.0853,  0.0992,
          0.0186,  0.0153]], grad_fn=<AddmmBackward0>)

In [10]:
# 在这个例子中，我们提供了一个简化的实现。
# 由于Sequential类本质上是一个OrderedDict容器，我们可以用它的add_module函数来添加一个块。
# 这在实现MySequential类时非常有用。
net = nn.Sequential()
net.add_module('hidden', nn.Linear(20, 256))
net.add_module('relu', nn.ReLU())
net.add_module('output', nn.Linear(256, 10))
net(X)

tensor([[-0.1105, -0.1595,  0.0608,  0.0813,  0.0857,  0.0167, -0.1526,  0.0688,
          0.1065, -0.0058],
        [-0.1836, -0.1531,  0.1073,  0.0129,  0.0446, -0.1036, -0.0503, -0.0502,
          0.0540, -0.1891]], grad_fn=<AddmmBackward0>)